In [36]:
import numpy as np
import torch
from torch.utils import data

In [37]:
def synthetic_data(w, b, num_examples):
    
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    
    return X, y.reshape(-1, 1)
    
    
true_w = torch.tensor([2, -3.4])
true_b = 4.2

features, labels = synthetic_data(true_w, true_b, 1000)

In [38]:
def load_array(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset,batch_size,shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

#next(iter(data_iter))

In [39]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)


loss = nn.MSELoss()


trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [40]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch}, loss {l:f}')

w = net[0].weight.data
b = net[0].bias.data
print('w的估计误差:', true_w - w.reshape(true_w.shape))
print('b的估计误差:', true_b - b)

epoch 0, loss 0.000350
epoch 1, loss 0.000100
epoch 2, loss 0.000100
w的估计误差: tensor([-0.0002,  0.0015])
b的估计误差: tensor([0.0002])
